## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-20-01-18-11 +0000,nypost,COO accused of stealing $132K in public cash a...,https://nypost.com/2026/02/19/us-news/amy-knox...
1,2026-02-20-01-16-40 +0000,nyt,MAHA Moms Turn Against Trump: ‘Women Feel Like...,https://www.nytimes.com/2026/02/19/us/politics...
2,2026-02-20-01-15-54 +0000,nyt,Two Sisters Died in Tahoe Area Avalanche Durin...,https://www.nytimes.com/2026/02/19/us/tahoe-av...
3,2026-02-20-01-15-26 +0000,bbc,Two sisters among those killed in Lake Tahoe a...,https://www.bbc.com/news/articles/c4g02nxx2plo...
4,2026-02-20-01-15-17 +0000,nyt,"With ‘Tremendous’ Deals at Stake, Trump Is Bri...",https://www.nytimes.com/2026/02/19/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2429/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,53
103,andrew,32
69,epstein,29
109,iran,27
102,prince,27


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
158,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...,179
81,2026-02-19-23-00-14 +0000,nyt,British Police Arrest Former Prince Andrew Ami...,https://www.nytimes.com/2026/02/19/world/europ...,177
151,2026-02-19-20-28-03 +0000,nyt,The police are investigating former Prince And...,https://www.nytimes.com/live/2026/02/19/world/...,163
172,2026-02-19-19-31-44 +0000,nypost,Ex-Prince Andrew looks aghast in first photos ...,https://nypost.com/2026/02/19/world-news/ex-pr...,149
303,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...,146


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
158,179,2026-02-19-20-12-03 +0000,nypost,Trump expresses sympathy for the royal family ...,https://nypost.com/2026/02/19/us-news/trump-ex...
359,93,2026-02-19-07-49-29 +0000,nypost,Former South Korean President Yoon sentenced t...,https://nypost.com/2026/02/19/world-news/forme...
303,81,2026-02-19-11-48-00 +0000,wsj,"As Trump’s Board of Peace meets Thursday, can ...",https://www.wsj.com/politics/trump-moves-towar...
259,74,2026-02-19-14-39-21 +0000,nypost,White House warns Iran it’d be ‘very wise’ to ...,https://nypost.com/2026/02/19/us-news/white-ho...
48,72,2026-02-20-00-00-00 +0000,wsj,The husband of embattled Labor Secretary Lori ...,https://www.wsj.com/politics/policy/labor-secr...
295,54,2026-02-19-12-11-49 +0000,nypost,UK Prime Minister Keir Starmer warned ‘nobody ...,https://nypost.com/2026/02/19/world-news/uk-pr...
369,52,2026-02-19-05-01-09 +0000,nyt,Policy Flip-Flops Hurt the British Leader. The...,https://www.nytimes.com/2026/02/19/world/europ...
244,50,2026-02-19-15-39-00 +0000,wsj,"The U.S. trade deficit jumped in December, the...",https://www.wsj.com/economy/trade/u-s-trade-de...
220,47,2026-02-19-17-07-21 +0000,nypost,Chaos at major AI summit as scandal-plagued Bi...,https://nypost.com/2026/02/19/business/bill-ga...
52,43,2026-02-19-23-55-02 +0000,nypost,"Two sisters, former radio exec among Lake Taho...",https://nypost.com/2026/02/19/us-news/photos-l...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
